In [2]:
import numpy as np
import seaborn as sns
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [4]:
df=sns.load_dataset('tips')

In [5]:
df.shape

(244, 7)

In [6]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [40]:
cat_columns=[column for column in df.columns if df[column].dtypes=='category']

['sex', 'smoker', 'day', 'time']

In [ ]:
df[]

In [7]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [10]:
x=df.iloc[:,1:]

In [13]:
y=df.iloc[:,0]

In [14]:
y

0      16.99
1      10.34
2      21.01
3      23.68
4      24.59
       ...  
239    29.03
240    27.18
241    22.67
242    17.82
243    18.78
Name: total_bill, Length: 244, dtype: float64

In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=2)

In [16]:
#numerical Pipelining
numeric_preprocessor = Pipeline(
    steps=[
        ("imputation_mean", SimpleImputer(missing_values=np.nan, strategy="mean")),
        ("scaler", StandardScaler()),
    ]
)

In [17]:
numeric_preprocessor

Pipeline(steps=[('imputation_mean', SimpleImputer()),
                ('scaler', StandardScaler())])

In [18]:
#Categorical pipelining
categorical_preprocessor=Pipeline(steps=[
        (
            "imputation_constant",
            SimpleImputer(fill_value="missing", strategy="constant"),
        ),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)

In [19]:
categorical_preprocessor

Pipeline(steps=[('imputation_constant',
                 SimpleImputer(fill_value='missing', strategy='constant')),
                ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [20]:
preprocessor=Pipeline(
    steps=[("categorical",categorical_preprocessor,["sex","smoker","day","time"]),("numerical",numeric_preprocessor)]

)

In [21]:
preprocessor

Pipeline(steps=[('categorical',
                 Pipeline(steps=[('imputation_constant',
                                  SimpleImputer(fill_value='missing',
                                                strategy='constant')),
                                 ('onehot',
                                  OneHotEncoder(handle_unknown='ignore'))])),
                ('numerical',
                 Pipeline(steps=[('imputation_mean', SimpleImputer()),
                                 ('scaler', StandardScaler())]))])

In [22]:
pipe=Pipeline(
    [("preprocessor",preprocessor),("regressor",RandomForestRegressor())]

)

In [23]:
pipe

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('categorical',
                                  Pipeline(steps=[('imputation_constant',
                                                   SimpleImputer(fill_value='missing',
                                                                 strategy='constant')),
                                                  ('onehot',
                                                   OneHotEncoder(handle_unknown='ignore'))])),
                                 ('numerical',
                                  Pipeline(steps=[('imputation_mean',
                                                   SimpleImputer()),
                                                  ('scaler',
                                                   StandardScaler())]))])),
                ('regressor', RandomForestRegressor())])

In [24]:
pipe.fit(x_train,y_train)

ValueError: Cannot center sparse matrices: pass `with_mean=False` instead. See docstring for motivation and alternatives.